In [ ]:
# 출처 : https://wikidocs.net/233344
# LangChain 설치 및 업데이트
#!pip install -U langchain langchain-community langchain-experimental langchain-core langchain-openai langsmith langchainhub python-dotenv unstructured chromadb faiss-cpu rank_bm25 python-docx sqlalchemy

In [ ]:
# 루트경로에 .env 파일을 만들고, OPENAI_API_KEY='{API_KEY}' 식으로 입력한다.
# API 키를 환경변수로 관리하기 위한 .env설정 파일 로딩
import os
from dotenv import load_dotenv

load_dotenv() # API 키 정보 로드
print(f"[API KEY]\n{os.environ['OPENAI_API_KEY']}")

In [2]:
from langchain import hub
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import JsonOutputParser
from langchain.document_loaders import WebBaseLoader, TextLoader
from langchain.callbacks.base import BaseCallbackHandler

prompt = hub.pull("teddynote/chain-of-density-korean")
prompt

ChatPromptTemplate(input_variables=['ARTICLE'], metadata={'lc_hub_owner': 'teddynote', 'lc_hub_repo': 'chain-of-density-korean', 'lc_hub_commit_hash': 'f7122cae5cd37900b29543353cb15dbd8dd8f0a6fd20c4413a9a958c19b800c9'}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['ARTICLE'], template='Article: {ARTICLE}\nYou will generate increasingly concise, entity-dense summaries of the above article. \n\nRepeat the following 2 steps 5 times. \n\nStep 1. Identify 1-3 informative entities (";" delimited) from the article which are missing from the previously generated summary. \nStep 2. Write a new, denser summary of identical length which covers every entity and detail from the previous summary plus the missing entities. \n\nA missing entity is:\n- relevant to the main story, \n- specific yet concise (100 words or fewer), \n- novel (not in the previous summary), \n- faithful (present in the article), \n- anywhere (can be located anywhere in the article).\n\nGuidelines

In [4]:
# Load some data to summarize

#loader = WebBaseLoader("https://www.aitimes.com/news/articleView.html?idxno=131777")
#docs = loader.load()
#content = docs[0].page_content

# ./data 폴더에 있는 .txt 확장자 파일을 모두 불러옴.
data_folder = "./data"
docs = []
content:str = ''

for filename in os.listdir(data_folder):
    if filename.endswith(".txt"):
        file_path = os.path.join(data_folder, filename)
        loader = TextLoader(file_path)
        docs.extend(loader.load())

print(f"*문서의 수: {len(docs)}\n")
for doc in docs:
    content += doc.page_content
print(f'*context 길이: {len(content)}')
print("\n========= [앞부분] 미리보기 =========\n")    
print(content[:100])

class StreamCallback(BaseCallbackHandler):
    def on_llm_new_token(self, token, **kwargs):
        print(token, end="", flush=True)


# Create the chain, including
chain = (
    prompt
    | ChatOpenAI(
        temperature=0,
        model="gpt-3.5-turbo",
        streaming=True,
        callbacks=[StreamCallback()],
    )
    | JsonOutputParser()
    | (lambda x: x[-1]["Denser_Summary"])
)

# Invoke the chain
result = chain.invoke({"ARTICLE": content})
print(result)

*문서의 수: 2

*context 길이: 4068

========= [앞부분] 미리보기 =========

경조사 지원 규정
1. 적용대상
계약직 사원을 포함한 임직원(고문 및 용역은 사장이 별도로 결정)
2. 경조사 지원기준
구 분
내 역
휴가(일)
금액(원)
기타
경 사
본인 결혼

[
    {
        "Missing_Entities": "결혼퇴직",
        "Denser_Summary": "이 규정은 2007년 9월 14일부터 시행되며, 결혼퇴직의 경우 퇴직 1개월 이내에 결혼할 시에는 경조금을 지급한다. 경조금 신청 시 휴가 신청도 동시에 진행해야 하며, 특별한 경우 사업부장 합의 시 경조금 신청일 이후 6개월 이내에 신청 가능하다. 부모 회갑, 배우자 부모회갑, 부모고희, 배우자부모고희 등의 경우에도 경조금이 지급된다."
    },
    {
        "Missing_Entities": "신규채용",
        "Denser_Summary": "월급여는 매월 20일 지급되며, 신규채용 및 복직자의 급여는 발령일로부터 일할계산된다. 신규채용되어 수습기간중인 자는 월급여의 90%를 받으며, 복직자의 급여는 휴직당시 직급·호봉에 해당하는 현급여액을 받는다. 또한, 전임·파견, 복귀자의 급여는 발령일을 기준으로 매월 20일까지 신근무지에서, 21일 이후에는 전근무지에서 지급된다."
    },
    {
        "Missing_Entities": "퇴직자",
        "Denser_Summary": "퇴직자의 경우 퇴직명령전일까지의 급여를 일할계산하여 지급하며, 퇴직금은 만 1년이상 근속한 직원이 퇴직할 경우에 지급된다. 퇴직금은 근속년수에 따라 평균임금의 30일분을 지급하며, 등기임원으로 선임된 경우에는 발령일을 기준으로 퇴직금을 계산하여 지급한다. 또한, 퇴직금은 평균임금 × 30 × 근속기간 / 365에 의해 계산되어 원단위에서 절사된다."
    },
    {
        "Missing_Entities"